In [2]:
%load_ext autoreload
%autoreload 2

# Path to the main CAMELS folder
CAMELS_DIR = "/home/jhbai/ealstm_regional_modeling/data/CAMELS/"

# Path to the main directory of this repository
BASE_CODE_DIR = "/home/jhbai/ealstm_regional_modeling"

In [5]:
# Imports
import json
import torch
import pickle
import sys
from collections import defaultdict
from pathlib import Path

import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bokeh.sampledata import us_states

sys.path.append(BASE_CODE_DIR)
from main import Model
from papercode import datautils
from papercode.clusterutils import get_silhouette_scores, get_clusters, get_label_2_color, get_variance_reduction
from papercode.datasets import CamelsTXT
from papercode.datautils import load_attributes
from papercode.plotutils import get_shape_collections
from papercode.utils import get_basin_list


In [7]:
# Convert file/folder path to pathlib Path objects
CAMELS_DIR = Path(CAMELS_DIR)
CODE_DIR = Path(BASE_CODE_DIR)

# Not really of importance, but need to be defined for the data set class.
VAL_START = pd.to_datetime('01101989', format='%d%m%Y')
VAL_END = pd.to_datetime('30091999', format='%d%m%Y')
GRID_SIZE=0.125

In [4]:
basins = get_basin_list()

NameError: name 'get_basin_list' is not defined

In [7]:
#datautils.add_camels_attributes(CAMELS_DIR)

In [53]:
# draw the maps
from shapely.ops import unary_union
from shapely.geometry import Polygon
import folium
from pyproj import CRS, Transformer

shapefile = datautils.load_shape(CAMELS_DIR)

In [56]:
#creates the projections used to normalize location values
incrs = CRS.from_proj4("+proj=aea +lon_0=-96 +lat_1=29.5 +lat_2=45.5 +lat_0=23 +datum=NAD83")
crs_4326 = CRS.from_epsg(4326)
transformer = Transformer.from_crs(incrs, crs_4326)

In [71]:
#generates visualization of map 
m = folium.Map(width=650, height=500, zoom_start=4, 
                location=[41.960781097662604, -78.16963195732737],              
                tiles='Stamen Terrain')
shapefile1=shapefile[0:200]
new_shape=datautils.create_shape(shapefile, '02422500',transformer)
folium.GeoJson(new_shape,name='02046000',tooltip='02046000').add_to(m)

In [72]:
m

In [342]:
#calculate data using gridded data 
filterlist=datautils.list_overlap(new_shape,CODE_DIR,GRID_SIZE/2)
result=datautils.find_intersect(new_shape,filterlist)
result['wprec'] = result['weight']*result['precipitation']

g=result.groupby( ['Year', 'Mnth', 'Day'])
final=g.sum()

/home/jhbai/ealstm_regional_modeling/data/east/data_32.9375_-86.5625
/home/jhbai/ealstm_regional_modeling/data/east/data_32.6875_-86.5625
/home/jhbai/ealstm_regional_modeling/data/east/data_32.6875_-86.8125
/home/jhbai/ealstm_regional_modeling/data/east/data_32.9375_-86.8125
/home/jhbai/ealstm_regional_modeling/data/east/data_32.6875_-86.6875
/home/jhbai/ealstm_regional_modeling/data/east/data_32.9375_-86.6875
/home/jhbai/ealstm_regional_modeling/data/east/data_32.5625_-86.9375
/home/jhbai/ealstm_regional_modeling/data/east/data_32.8125_-86.9375
/home/jhbai/ealstm_regional_modeling/data/east/data_32.8125_-86.8125
/home/jhbai/ealstm_regional_modeling/data/east/data_32.5625_-86.8125
/home/jhbai/ealstm_regional_modeling/data/east/data_32.5625_-86.5625
/home/jhbai/ealstm_regional_modeling/data/east/data_32.8125_-86.5625
/home/jhbai/ealstm_regional_modeling/data/east/data_32.9375_-86.9375
/home/jhbai/ealstm_regional_modeling/data/east/data_32.6875_-86.9375
/home/jhbai/ealstm_regional_modeli

In [66]:
#read camels data
basins = get_basin_list()
forcings = {}
discharge = {}
for basin in basins:
    forcings[basin] = datautils.load_forcing(CAMELS_DIR, basin)
    discharge[basin] = datautils.load_discharge(CAMELS_DIR, basin, forcings[basin][1])

In [65]:
#xperimental
import torch
import re
maurier=datautils.list_grid(CODE_DIR)
arr=np.ndarray(shape=(22645,400,400,5), dtype=float)

from numpy import zeros, newaxis
bigl=[]
for square in maurier[0:2]:
    x=int((float(re.findall(r"-?\d+.\d+", square.name)[1])+67.0625)*-8)
    y=int((float(re.findall(r"-?\d+.\d+", square.name)[0])-25.1875)*8)
    print(square)
    sq=datautils.read_grid(square,1,True)
    p=sq.values
    l=[]
    counter=0
    for row in p:
        b = row[newaxis, 3:8]
        l.append(b)
        arr[counter][x][y]=b
        counter+=1
    persquare=np.vstack(l)[:,newaxis]
    bigl.append(persquare)
res=np.hstack(bigl)
res.shape
arr[1][x][y]

/home/jhbai/ealstm_regional_modeling/data/east/data_41.1875_-73.8125
/home/jhbai/ealstm_regional_modeling/data/east/data_31.0625_-89.8125


array([ 0.  , 18.67,  2.91,  6.24,  1.  ])